In [1]:
#!/usr/bin/env python3

"""
Demonstrates how to:
    * use the MAML wrapper for fast-adaptation,
    * use the benchmark interface to load mini-ImageNet, and
    * sample tasks and split them in adaptation and evaluation sets.

To contrast the use of the benchmark interface with directly instantiating mini-ImageNet datasets and tasks, compare with `protonet_miniimagenet.py`.
"""

import random
import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt
import learn2learn as l2l
from learn2learn.data.transforms import (NWays,
                                         KShots,
                                         LoadData,
                                         RemapLabels,
                                         ConsecutiveLabels)
import os

In [ ]:


def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        adaptation_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(adaptation_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    evaluation_error = loss(predictions, evaluation_labels)
    evaluation_accuracy = accuracy(predictions, evaluation_labels)
    return evaluation_error, evaluation_accuracy


def perform_experiment(dataset,
        ways=5,
        shots=5,
        meta_lr=0.003,
        fast_lr=0.5,
        meta_batch_size=32,
        adaptation_steps=1,
        num_iterations=60000,
        cuda=True,
        seed=42,
):
    
    Meta_Train_Accuracy = []
    Meta_Train_Error = []
    Meta_Val_Accuracy = []
    Meta_Val_Error = []
#     Iterations = []
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    device = torch.device('cpu')
    if cuda and torch.cuda.device_count():
        torch.cuda.manual_seed(seed)
        device = torch.device('cuda')

    # Create Tasksets using the benchmark interface
    tasksets = l2l.vision.benchmarks.get_tasksets(dataset,
                                                  train_samples=2*shots,
                                                  train_ways=ways,
                                                  test_samples=2*shots,
                                                  test_ways=ways,
                                                  root='~/data',
    )

    # Create model
    model = l2l.vision.models.MiniImagenetCNN(ways)
    model.to(device)
    maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
    opt = optim.Adam(maml.parameters(), meta_lr)
    loss = nn.CrossEntropyLoss(reduction='mean')

    for iteration in range(num_iterations):
        opt.zero_grad()
        meta_train_error = 0.0
        meta_train_accuracy = 0.0
        meta_valid_error = 0.0
        meta_valid_accuracy = 0.0
        for task in range(meta_batch_size):
            # Compute meta-training loss
            learner = maml.clone()
            batch = tasksets.train.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               loss,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            evaluation_error.backward()
            meta_train_error += evaluation_error.item()
            meta_train_accuracy += evaluation_accuracy.item()

            # Compute meta-validation loss
            learner = maml.clone()
            batch = tasksets.validation.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               loss,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            meta_valid_error += evaluation_error.item()
            meta_valid_accuracy += evaluation_accuracy.item()

        # Print some metrics
#         print('\n')
#         print('Iteration', iteration)
#         print('Meta Train Error', meta_train_error / meta_batch_size)
#         print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
#         print('Meta Valid Error', meta_valid_error / meta_batch_size)
#         print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)
        
        meta_train_error =  meta_train_error / meta_batch_size
        meta_train_accuracy = meta_train_accuracy / meta_batch_size
        meta_val_error =  meta_valid_error / meta_batch_size
        meta_val_accuracy = meta_valid_accuracy / meta_batch_size
        if(iteration % 32 ==0):
            print('\n')
            print('Iteration', iteration)
            print('Meta Train Error', meta_train_error)
            print('Meta Train Accuracy', meta_train_accuracy)
            print('Meta Valid Error', meta_val_error)
            print('Meta Valid Accuracy', meta_val_accuracy)

        Meta_Train_Accuracy.append(meta_train_accuracy)
        Meta_Train_Error.append(meta_train_error)
        Meta_Val_Accuracy.append(meta_val_accuracy)
        Meta_Val_Error.append(meta_val_error)

        # Average the accumulated gradients and optimize
        for p in maml.parameters():
            p.grad.data.mul_(1.0 / meta_batch_size)
        opt.step()

    meta_test_error = 0.0
    meta_test_accuracy = 0.0
    for task in range(meta_batch_size):
        # Compute meta-testing loss
        learner = maml.clone()
        batch = tasksets.test.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                           learner,
                                                           loss,
                                                           adaptation_steps,
                                                           shots,
                                                           ways,
                                                           device)
        meta_test_error += evaluation_error.item()
        meta_test_accuracy += evaluation_accuracy.item()
#     print('Meta Test Error', meta_test_error / meta_batch_size)
#     print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
    Meta_Test_Error = meta_test_error / meta_batch_size
    print('Meta Test Error', Meta_Test_Error)
    Meta_Test_Accuracy = meta_test_accuracy / meta_batch_size
    print('Meta Test Accuracy', Meta_Test_Accuracy)
    
    if not os.path.exists('plots'):
        os.makedirs('plots')
    if not os.path.exists('plots/acc'):
        os.makedirs('plots/acc')
    if not os.path.exists('plots/loss'):
        os.makedirs('plots/loss')
        
    ###### Plot Accuracies ######
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.plot(list(range(0, len(Meta_Train_Accuracy))), Meta_Train_Accuracy, label="Meta Train ")
    ax.plot(list(range(0, len(Meta_Val_Accuracy))), Meta_Val_Accuracy, label="Meta Val")
#     ax.text((len(Meta_Val_Accuracy)/2), 0, 'Meta Test:{0}'.format(round(meta_test_accuracy, 2)), style='italic',
#         bbox={'facecolor': 'red', 'alpha': 0.25, 'pad': 5})
    # place a text box in upper left in axes coords
    ax.text(0.05, 0.5, 'Meta Test:{0}'.format(round(Meta_Test_Accuracy, 2)), transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)
    plt.title('Adaption')
    plt.xlabel('Iteration')
    plt.ylabel('Meta Accuracy')
    ax.legend()
    plt.savefig('./plots/acc/{0}_ways_{1}_shots_{2}_Acc_I_{3}.png'.format(dataset, ways, shots, num_iterations),
               dpi=150)
    ###### Plot Accuracies ######

    ###### Plot Errors ######
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.plot(list(range(0, len(Meta_Train_Error))), Meta_Train_Error, label="Meta Train ")
    ax.plot(list(range(0, len(Meta_Val_Error))), Meta_Val_Accuracy, label="Meta Val Lss")
#     ax.text((len(Meta_Val_Accuracy)/2),0, 'Meta Test Accuracy:{0}  Meta Test Loss:{1}'.format(round(meta_test_accuracy, 2),
#                               round(meta_test_error, 2)), style='italic',
#         bbox={'facecolor': 'red', 'alpha': 0.25, 'pad': 5})
    ax.text(0.05, 0.5, 'MTestAcc:{0}  MTestLoss:{1}'.format(round(Meta_Test_Accuracy, 2),
                                           round(Meta_Test_Error, 2)), transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)
    plt.title('Adaption')
    plt.xlabel('Iteration')
    plt.ylabel('Meta Loss')
    ax.legend()
    plt.savefig('./plots/loss/{0}_ways_{1}_shots_{2}_Loss_I_{3}.png'.format(dataset, ways, shots, num_iterations), dpi=150)
    ###### Plot Errors ######


if __name__ == '__main__':
    Ways = [5]
    Shots = [5, 1]
    Iterations = [10000, 60000, 120000]
    for ways in Ways:
        for shots in Shots:
            for iteration in Iterations:
                
                perform_experiment(dataset='mini-imagenet',
                ways=ways,
                shots=shots,
                meta_lr=0.003,
                fast_lr=0.5,
                meta_batch_size=32,
                adaptation_steps=1,
                num_iterations=iteration,
                cuda=True,
                seed=42)



Iteration 0
Meta Train Error 6.996662676334381
Meta Train Accuracy 0.21749999094754457
Meta Valid Error 7.403078213334084
Meta Valid Accuracy 0.21374999172985554


Iteration 32
Meta Train Error 1.482303410768509
Meta Train Accuracy 0.3637499907054007
Meta Valid Error 1.5097931288182735
Meta Valid Accuracy 0.34249999141320586


Iteration 64
Meta Train Error 1.479987669736147
Meta Train Accuracy 0.3562499894760549
Meta Valid Error 1.4676995314657688
Meta Valid Accuracy 0.34874998894520104


Iteration 96
Meta Train Error 1.404117964208126
Meta Train Accuracy 0.4024999891407788
Meta Valid Error 1.4025105759501457
Meta Valid Accuracy 0.3737499916460365


Iteration 128
Meta Train Error 1.3663399778306484
Meta Train Accuracy 0.41624998580664396
Meta Valid Error 1.4416313283145428
Meta Valid Accuracy 0.40499998815357685


Iteration 160
Meta Train Error 1.403459396213293
Meta Train Accuracy 0.40749998623505235
Meta Valid Error 1.4031923078000546
Meta Valid Accuracy 0.4012499866075814


Iterat



Iteration 1600
Meta Train Error 0.9229499204084277
Meta Train Accuracy 0.6199999880045652
Meta Valid Error 1.1074733138084412
Meta Valid Accuracy 0.5799999851733446


Iteration 1632
Meta Train Error 0.8381934575736523
Meta Train Accuracy 0.691249979659915
Meta Valid Error 1.0735466200858355
Meta Valid Accuracy 0.5799999851733446


Iteration 1664
Meta Train Error 0.8398129660636187
Meta Train Accuracy 0.6762499818578362
Meta Valid Error 1.0275939349085093
Meta Valid Accuracy 0.5749999852851033


Iteration 1696
Meta Train Error 0.9055212773382664
Meta Train Accuracy 0.648749983869493
Meta Valid Error 1.079764787107706
Meta Valid Accuracy 0.5674999859184027


Iteration 1728
Meta Train Error 0.8314317921176553
Meta Train Accuracy 0.6874999850988388
Meta Valid Error 1.0743969306349754
Meta Valid Accuracy 0.5762499896809459


Iteration 1760
Meta Train Error 0.9161848891526461
Meta Train Accuracy 0.6412499863654375
Meta Valid Error 1.0957323759794235
Meta Valid Accuracy 0.5787499845027924





Iteration 3200
Meta Train Error 0.8162607969716191
Meta Train Accuracy 0.6774999778717756
Meta Valid Error 1.0391240268945694
Meta Valid Accuracy 0.5787499900907278


Iteration 3232
Meta Train Error 0.8042523777112365
Meta Train Accuracy 0.693749981932342
Meta Valid Error 1.0289470087736845
Meta Valid Accuracy 0.6174999820068479


Iteration 3264
Meta Train Error 0.7236395636573434
Meta Train Accuracy 0.7299999790266156
Meta Valid Error 0.9826140478253365
Meta Valid Accuracy 0.6137499837204814


Iteration 3296
Meta Train Error 0.7600036766380072
Meta Train Accuracy 0.7049999795854092
Meta Valid Error 1.031401064246893
Meta Valid Accuracy 0.5949999839067459


Iteration 3328
Meta Train Error 0.7551403883844614
Meta Train Accuracy 0.7124999864026904
Meta Valid Error 0.9952814495190978
Meta Valid Accuracy 0.5987499868497252


Iteration 3360
Meta Train Error 0.8181958552449942
Meta Train Accuracy 0.684999980032444
Meta Valid Error 1.09951813519001
Meta Valid Accuracy 0.5399999842047691


I



Iteration 4800
Meta Train Error 0.731460995040834
Meta Train Accuracy 0.6962499832734466
Meta Valid Error 1.0651500709354877
Meta Valid Accuracy 0.5899999886751175


Iteration 4832
Meta Train Error 0.6933429343625903
Meta Train Accuracy 0.7362499795854092
Meta Valid Error 1.0179973393678665
Meta Valid Accuracy 0.588749990798533


Iteration 4864
Meta Train Error 0.6571851000189781
Meta Train Accuracy 0.7562499763444066
Meta Valid Error 1.0376825500279665
Meta Valid Accuracy 0.5824999818578362


Iteration 4896
Meta Train Error 0.7414779937826097
Meta Train Accuracy 0.7099999841302633
Meta Valid Error 0.9625277575105429
Meta Valid Accuracy 0.6524999858811498


Iteration 4928
Meta Train Error 0.7240946302190423
Meta Train Accuracy 0.7374999783933163
Meta Valid Error 0.9570359848439693
Meta Valid Accuracy 0.6387499840930104


Iteration 4960
Meta Train Error 0.7288475167006254
Meta Train Accuracy 0.7287499727681279
Meta Valid Error 0.9574083164334297
Meta Valid Accuracy 0.6312499875202775




Iteration 6400
Meta Train Error 0.7173977363854647
Meta Train Accuracy 0.7199999839067459
Meta Valid Error 1.0453412104398012
Meta Valid Accuracy 0.5862499866634607


Iteration 6432
Meta Train Error 0.741500592790544
Meta Train Accuracy 0.7174999760463834
Meta Valid Error 1.003321098163724
Meta Valid Accuracy 0.6224999809637666


Iteration 6464
Meta Train Error 0.6664000544697046
Meta Train Accuracy 0.722499979659915
Meta Valid Error 0.8954232428222895
Meta Valid Accuracy 0.6499999817460775


Iteration 6496
Meta Train Error 0.6700297384522855
Meta Train Accuracy 0.7337499810382724
Meta Valid Error 0.9312158189713955
Meta Valid Accuracy 0.6562499785795808


Iteration 6528
Meta Train Error 0.7093068619724363
Meta Train Accuracy 0.7337499829009175
Meta Valid Error 1.125365149229765
Meta Valid Accuracy 0.5612499844282866


Iteration 6560
Meta Train Error 0.6549635725095868
Meta Train Accuracy 0.7562499772757292
Meta Valid Error 1.0097574414685369
Meta Valid Accuracy 0.6324999863281846




...



Iteration 19936
Meta Train Error 0.616471063811332
Meta Train Accuracy 0.7649999763816595
Meta Valid Error 1.017876423895359
Meta Valid Accuracy 0.5837499825283885


Iteration 19968
Meta Train Error 0.6218304201029241
Meta Train Accuracy 0.7749999770894647
Meta Valid Error 1.085621615871787
Meta Valid Accuracy 0.5787499854341149


Iteration 20000
Meta Train Error 0.6022961181588471
Meta Train Accuracy 0.7612499808892608
Meta Valid Error 1.0523810125887394
Meta Valid Accuracy 0.5649999883025885


Iteration 20032
Meta Train Error 0.6019211793318391
Meta Train Accuracy 0.7899999767541885
Meta Valid Error 1.0320780128240585
Meta Valid Accuracy 0.6012499872595072


Iteration 20064
Meta Train Error 0.6621249271556735
Meta Train Accuracy 0.748749976977706
Meta Valid Error 1.054237999022007
Meta Valid Accuracy 0.5749999871477485


Iteration 20096
Meta Train Error 0.5792635874822736
Meta Train Accuracy 0.7787499744445086
Meta Valid Error 1.043490458279848
Meta Valid Accuracy 0.586249983869493



Iteration 21504
Meta Train Error 0.5695994421839714
Meta Train Accuracy 0.7687499765306711
Meta Valid Error 0.9907091781497002
Meta Valid Accuracy 0.622499986551702


Iteration 21536
Meta Train Error 0.6582533591426909
Meta Train Accuracy 0.7499999776482582
Meta Valid Error 0.9649194087833166
Meta Valid Accuracy 0.6537499846890569


Iteration 21568
Meta Train Error 0.5671187434345484
Meta Train Accuracy 0.7862499803304672
Meta Valid Error 1.0678951293230057
Meta Valid Accuracy 0.5899999886751175


Iteration 21600
Meta Train Error 0.6004813057370484
Meta Train Accuracy 0.7737499810755253
Meta Valid Error 1.0674699377268553
Meta Valid Accuracy 0.5949999867007136


Iteration 21632
Meta Train Error 0.533840290736407
Meta Train Accuracy 0.7974999751895666
Meta Valid Error 1.0383614301681519
Meta Valid Accuracy 0.5949999876320362


Iteration 21664
Meta Train Error 0.5911975386552513
Meta Train Accuracy 0.777499983087182
Meta Valid Error 1.0245815385133028
Meta Valid Accuracy 0.596249984577



Iteration 23072
Meta Train Error 0.6032536677084863
Meta Train Accuracy 0.782499979250133
Meta Valid Error 1.027675548568368
Meta Valid Accuracy 0.597499979659915


Iteration 23104
Meta Train Error 0.5326941111125052
Meta Train Accuracy 0.8012499753385782
Meta Valid Error 1.0277731362730265
Meta Valid Accuracy 0.5824999874457717


Iteration 23136
Meta Train Error 0.5574228558689356
Meta Train Accuracy 0.7962499782443047
Meta Valid Error 0.9681378472596407
Meta Valid Accuracy 0.6112499833106995


Iteration 23168
Meta Train Error 0.4889708627015352
Meta Train Accuracy 0.8174999793991446
Meta Valid Error 1.0003505544736981
Meta Valid Accuracy 0.6074999868869781


Iteration 23200
Meta Train Error 0.5856312508694828
Meta Train Accuracy 0.7812499795109034
Meta Valid Error 1.0030281394720078
Meta Valid Accuracy 0.5849999878555536


Iteration 23232
Meta Train Error 0.5380262611433864
Meta Train Accuracy 0.7924999799579382
Meta Valid Error 0.9860846605151892
Meta Valid Accuracy 0.628749985247



Iteration 24640
Meta Train Error 0.5944278188981116
Meta Train Accuracy 0.7774999793618917
Meta Valid Error 1.0065502896904945
Meta Valid Accuracy 0.6112499861046672


Iteration 24672
Meta Train Error 0.6678065778687596
Meta Train Accuracy 0.7499999757856131
Meta Valid Error 1.0615000110119581
Meta Valid Accuracy 0.6024999851360917


Iteration 24704
Meta Train Error 0.6436461447738111
Meta Train Accuracy 0.7762499786913395
Meta Valid Error 1.0032033864408731
Meta Valid Accuracy 0.591249986551702


Iteration 24736
Meta Train Error 0.5954387239180505
Meta Train Accuracy 0.7774999793618917
Meta Valid Error 1.0826604310423136
Meta Valid Accuracy 0.5524999871850014


Iteration 24768
Meta Train Error 0.5561721916310489
Meta Train Accuracy 0.792499978095293
Meta Valid Error 0.9968113694339991
Meta Valid Accuracy 0.622499980032444


Iteration 24800
Meta Train Error 0.5874907751567662
Meta Train Accuracy 0.782499972730875
Meta Valid Error 1.041027631610632
Meta Valid Accuracy 0.58999998308718



Iteration 26208
Meta Train Error 0.588479689322412
Meta Train Accuracy 0.7774999793618917
Meta Valid Error 0.9499395582824945
Meta Valid Accuracy 0.6099999891594052


Iteration 26240
Meta Train Error 0.546069304458797
Meta Train Accuracy 0.7999999802559614
Meta Valid Error 1.1186295878142118
Meta Valid Accuracy 0.5587499886751175


Iteration 26272
Meta Train Error 0.6852275263518095
Meta Train Accuracy 0.7637499757111073
Meta Valid Error 1.077697684057057
Meta Valid Accuracy 0.563749979250133


Iteration 26304
Meta Train Error 0.5148991500027478
Meta Train Accuracy 0.7987499814480543
Meta Valid Error 1.0447135213762522
Meta Valid Accuracy 0.5999999837949872


Iteration 26336
Meta Train Error 0.6677116905339062
Meta Train Accuracy 0.761249978095293
Meta Valid Error 1.0722002424299717
Meta Valid Accuracy 0.5787499854341149


Iteration 26368
Meta Train Error 0.5508471075445414
Meta Train Accuracy 0.7899999767541885
Meta Valid Error 1.021132530644536
Meta Valid Accuracy 0.593749984167516